In [6]:
# Install required modules

# !pip install transformers ipywidgets --upgrade
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
# !pip install sentencepiece
# !pip install sacremoses
# !pip install lxml

In [7]:
from bs4 import BeautifulSoup, Tag, NavigableString
import re
import os
from transformers import pipeline

In [8]:
#Calling pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-hi")

In [9]:
#Translation

def translate(dir): 
    file1 = open(dir, "r", encoding="utf-8")
    lines = file1.read()
    lines = BeautifulSoup(lines, "lxml")
    lines = lines.prettify()
    file1.close()

    file2 = open(dir, "w", encoding="utf-8")
    file2.close()

    result = re.search('<body(.*)>', lines)
    data = lines.split("<body" + result.group(1) + '>', 1)

    file2 = open(dir, "a", encoding="utf-8")

    file2.write(data[0] + "\n" + "<body" + result.group(1) + ">" + "\n")
    lines = data[1].rsplit("</body>", 1)

    start = 0
    end = 0
    content = ""
    tag = True
    script = False

    for i in range(0, len(lines[0])):
        if lines[0][i] != ">" and tag == True:
            content = content + lines[0][i]
        #test
        elif lines[0][i] == ">" and tag == True:
            content = content + lines[0][i]
            file2.write(content)
            content = ""
            if lines[0][i - 1] == "t" and lines[0][i - 2] == "p" and lines[0][i - 3] == "i" and lines[0][i - 4] == "r" and lines[0][i - 5] == "c" and lines[0][i - 6] == "s" and lines[0][i - 7] == "/":
                script = False
            if lines[0][i + 1] != "<":
                if lines[0][i - 1] == "t" and lines[0][i - 2] == "p" and lines[0][i - 3] == "i" and lines[0][i - 4] == "r" and lines[0][i - 5] == "c" and lines[0][i - 6] == "s":
                    tag = True
                    script = True
                else:
                    tag = False
                    
        elif lines[0][i] == ">" and tag == False:
            content = content + lines[0][i]
        elif lines[0][i] != ">" and tag == False and lines[0][i] != "<" and script == False:
            content = content + lines[0][i]
        elif lines[0][i] == "<" and tag == True:
            file2.write(content)
            content = ""
            content = content + lines[0][i]
        elif lines[0][i] == "<" and tag == False and script == False:
            tag = True
            content = content.strip()
            if content != "":
                if "&amp;" in content:
                    position = content.find("&amp;")
                    translatedA = translator(content[:position])
                    translatedB = translator(content[position+4 :])
                    translation = str(translatedA[0]['translation_text']) + " &amp; " + str(translatedB[0]['translation_text'])
                    file2.write(translation)
                else:
                    if content != "":
                        translated = translator(content)
                        file2.write(str(translated[0]['translation_text']))
            content = ""
            content = content + lines[0][i]

    file2.write("</body>" + "\n" + lines[1])
    file1.close()
    file2.close()
    
    file1 = open(dir, "r", encoding="utf-8")
    lines = file1.read()
    lines = BeautifulSoup(lines, "lxml")
    lines = lines.prettify()
    file1.close()

    file1 = open(dir, "w", encoding="utf-8")
    file1.write(lines)
    file1.close()

In [11]:
# Iterating through files

rootdir = "websites"
path = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file

        if filepath.endswith(".html"):
            path.append(filepath)
count = 0
print(len(path))
print("*****TRANSLATION STARTED*****")
for dir in path:
    print(dir)
    count = count+1
    print(count)
    translate(dir)

print("*****TRANSLATION COMPLETED*****")
    


*****TRANSLATION STARTED*****
Outsite script
Outsite script
*****TRANSLATION COMPLETED*****
*****TRANSLATION COMPLETED*****
